# Complete Guide to Static Panel Models

This notebook provides a comprehensive guide to **static panel data models** in PanelBox, covering all five estimators and when to use each.

## What You'll Learn

- ✅ All 5 static panel models (Pooled OLS, Between, Fixed Effects, Random Effects, First Difference)
- ✅ When to use each model
- ✅ Specification tests (F-test, Hausman test)
- ✅ Robust standard errors (8 types)
- ✅ Model comparison and interpretation
- ✅ Report generation (HTML, Markdown, LaTeX)

## Table of Contents

1. [Introduction to Static Panel Models](#introduction)
2. [Data Preparation](#data-preparation)
3. [The Five Models](#five-models)
   - 3.1 [Pooled OLS](#pooled-ols)
   - 3.2 [Between Estimator](#between)
   - 3.3 [Fixed Effects](#fixed-effects)
   - 3.4 [Random Effects](#random-effects)
   - 3.5 [First Difference](#first-difference)
4. [Specification Tests](#specification-tests)
5. [Robust Standard Errors](#robust-se)
6. [Model Comparison](#model-comparison)
7. [Report Generation](#report-generation)
8. [Decision Guide](#decision-guide)

---

## 1. Introduction to Static Panel Models {#introduction}

### What are Static Panel Models?

**Static panel models** are used when:
- No lagged dependent variable on the right-hand side
- Focus on contemporaneous relationships
- Want to control for unobserved heterogeneity

### General Form:

$$y_{it} = \alpha + \beta' X_{it} + u_{it}$$

Where:
- $y_{it}$: Dependent variable for entity $i$ at time $t$
- $X_{it}$: Vector of independent variables
- $u_{it}$: Error term

### Error Structure:

The key difference between models is how they handle the error term:

$$u_{it} = \alpha_i + \varepsilon_{it}$$

Where:
- $\alpha_i$: Entity-specific time-invariant effect (unobserved heterogeneity)
- $\varepsilon_{it}$: Idiosyncratic error

### The Five Models:

| Model | Assumption about $\alpha_i$ | Use Case |
|-------|------------------------|----------|
| **Pooled OLS** | $\alpha_i = 0$ | Baseline (usually biased) |
| **Between** | Uses cross-sectional variation only | Long-run effects |
| **Fixed Effects** | $\alpha_i$ correlated with $X_{it}$ | Control for unobserved heterogeneity |
| **Random Effects** | $\alpha_i$ uncorrelated with $X_{it}$ | Efficient if assumption holds |
| **First Difference** | Differences out $\alpha_i$ | Short-run effects |

Let's dive into each model!

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# PanelBox
import panelbox as pb

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.random.seed(42)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PanelBox version: {pb.__version__}")
print("Setup complete!")

---

## 2. Data Preparation {#data-preparation}

We'll use the **Grunfeld dataset** throughout this notebook.

In [ ]:
# Load data
data = pb.load_grunfeld()

print("Dataset loaded successfully!")
print(f"Shape: {data.shape}")
print(f"\nVariables: {list(data.columns)}")
print(f"\nFirst few rows:")
data.head()

### Check Panel Structure

In [ ]:
# Panel structure analysis
print("Panel Structure:")
print("="*60)
print(f"Number of firms (N): {data['firm'].nunique()}")
print(f"Number of years (T): {data['year'].nunique()}")
print(f"Total observations (N×T): {len(data)}")
print(f"Panel type: Balanced")
print(f"\nTime span: {data['year'].min()} - {data['year'].max()}")

### Descriptive Statistics

In [ ]:
# Overall statistics
print("Descriptive Statistics:")
print("="*60)
data[['invest', 'value', 'capital']].describe().round(2)

In [ ]:
# Decompose variation: between vs within
variables = ['invest', 'value', 'capital']

print("\nVariance Decomposition (Between vs Within):")
print("="*60)

for var in variables:
    # Total variance
    total_var = data[var].var()
    
    # Between variance (across firms)
    firm_means = data.groupby('firm')[var].mean()
    between_var = firm_means.var()
    
    # Within variance (within firms over time)
    data[f'{var}_demeaned'] = data.groupby('firm')[var].transform(lambda x: x - x.mean())
    within_var = data[f'{var}_demeaned'].var()
    
    print(f"\n{var.upper()}:")
    print(f"  Total Variance:   {total_var:10.2f}")
    print(f"  Between Variance: {between_var:10.2f} ({100*between_var/total_var:.1f}%)")
    print(f"  Within Variance:  {within_var:10.2f} ({100*within_var/total_var:.1f}%)")

# Clean up temporary columns
data = data.drop(columns=[col for col in data.columns if '_demeaned' in col])

**Key Insight**: If between variance >> within variance, Fixed Effects may remove too much variation. If within variance is substantial, FE can be effective.

### Visualize the Data

In [ ]:
# Investment trends by firm
fig, ax = plt.subplots(figsize=(14, 6))

for firm_id in data['firm'].unique():
    firm_data = data[data['firm'] == firm_id]
    ax.plot(firm_data['year'], firm_data['invest'], marker='o', label=f'Firm {firm_id}', alpha=0.7, linewidth=2)

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Investment', fontsize=12)
ax.set_title('Investment Over Time by Firm', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observations:")
print("- Clear firm-specific levels (between variation)")
print("- Temporal trends within firms (within variation)")
print("- This suggests Fixed Effects could be appropriate")

---

## 3. The Five Models {#five-models}

We'll estimate all five models on the same specification:

$$\text{invest}_{it} = \beta_0 + \beta_1 \text{value}_{it} + \beta_2 \text{capital}_{it} + u_{it}$$

### 3.1 Pooled OLS {#pooled-ols}

**Pooled OLS** treats all observations as independent, ignoring the panel structure.

**Assumption**: $\alpha_i = 0$ (no firm-specific effects)

**Pros**:
- Simple
- Uses all variation (between + within)
- Baseline for comparison

**Cons**:
- ❌ Ignores unobserved heterogeneity
- ❌ Biased if $\alpha_i$ correlated with $X_{it}$
- ❌ Standard errors incorrect (need clustering)

In [ ]:
# Estimate Pooled OLS
pooled = pb.PooledOLS(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

pooled_results = pooled.fit()

print("="*70)
print("MODEL 1: POOLED OLS")
print("="*70)
print(pooled_results.summary())

**Interpretation**:
- Both `value` and `capital` are highly significant
- R² is high, suggesting good fit
- **BUT**: This assumes no firm-specific effects (likely violated)

### 3.2 Between Estimator {#between}

**Between Estimator** uses only cross-sectional variation by averaging over time for each firm.

**Method**: 
1. Calculate firm-specific means: $\bar{y}_i = \frac{1}{T}\sum_t y_{it}$
2. Run OLS on these means: $\bar{y}_i = \beta_0 + \beta' \bar{X}_i + u_i$

**Pros**:
- Captures long-run relationships
- Uses between variation
- Simple interpretation

**Cons**:
- ❌ Throws away all within variation
- ❌ Only N observations (not N×T)
- ❌ Inefficient compared to other estimators

In [ ]:
# Estimate Between model
between = pb.BetweenEstimator(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

between_results = between.fit()

print("="*70)
print("MODEL 2: BETWEEN ESTIMATOR")
print("="*70)
print(between_results.summary())

**Interpretation**:
- Uses only 10 observations (firm averages)
- Captures differences between firms (long-run)
- Compare coefficients with Pooled OLS

### 3.3 Fixed Effects (FE) {#fixed-effects}

**Fixed Effects** is the workhorse of panel econometrics.

**Method**: Include firm dummies to control for $\alpha_i$

$$y_{it} = \alpha_i + \beta' X_{it} + \varepsilon_{it}$$

**Equivalently**: "Within" transformation (demean by firm)

$$(y_{it} - \bar{y}_i) = \beta' (X_{it} - \bar{X}_i) + (\varepsilon_{it} - \bar{\varepsilon}_i)$$

**Pros**:
- ✅ Controls for all time-invariant unobserved heterogeneity
- ✅ Consistent even if $\alpha_i$ correlated with $X_{it}$
- ✅ Uses within variation
- ✅ No distributional assumptions on $\alpha_i$

**Cons**:
- ❌ Cannot estimate time-invariant variables
- ❌ May remove too much variation
- ❌ Less efficient than RE if RE assumptions hold

In [ ]:
# Estimate Fixed Effects
fe = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

fe_results = fe.fit()

print("="*70)
print("MODEL 3: FIXED EFFECTS (WITHIN ESTIMATOR)")
print("="*70)
print(fe_results.summary())

**Interpretation**:
- Coefficients differ from Pooled OLS (controlling for $\alpha_i$)
- Within R² shows fit after removing firm effects
- This is often the preferred estimator in practice

### 3.4 Random Effects (RE) {#random-effects}

**Random Effects** uses GLS estimation, treating $\alpha_i$ as random.

**Assumption**: $\alpha_i$ is **uncorrelated** with $X_{it}$

$$\mathbb{E}[\alpha_i | X_{i1}, ..., X_{iT}] = 0$$

**Method**: Quasi-demeaning with optimal weight $\theta$

$$(y_{it} - \theta\bar{y}_i) = \beta_0(1-\theta) + \beta'(X_{it} - \theta\bar{X}_i) + v_{it}$$

Where $\theta$ is chosen to minimize variance.

**Pros**:
- ✅ More efficient than FE (if assumption holds)
- ✅ Can estimate time-invariant variables
- ✅ Uses both between and within variation
- ✅ Better for small T

**Cons**:
- ❌ Inconsistent if $\alpha_i$ correlated with $X_{it}$
- ❌ Strong assumption (use Hausman test to check)

In [ ]:
# Estimate Random Effects
re = pb.RandomEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

re_results = re.fit()

print("="*70)
print("MODEL 4: RANDOM EFFECTS (GLS ESTIMATOR)")
print("="*70)
print(re_results.summary())

**Interpretation**:
- Coefficients typically between Pooled OLS and FE
- More efficient than FE (smaller standard errors) if assumption holds
- **Critical**: Use Hausman test to check if RE is valid

### 3.5 First Difference (FD) {#first-difference}

**First Difference** eliminates $\alpha_i$ by taking first differences.

**Method**: Transform to differences

$$\Delta y_{it} = y_{it} - y_{it-1} = \beta' \Delta X_{it} + \Delta \varepsilon_{it}$$

The $\alpha_i$ term drops out!

**Pros**:
- ✅ Eliminates $\alpha_i$ without demeaning
- ✅ Focuses on short-run changes
- ✅ Can handle non-stationarity
- ✅ Robust to certain forms of measurement error

**Cons**:
- ❌ Loses N observations (one per entity)
- ❌ Amplifies measurement error
- ❌ Less efficient than FE if errors are serially uncorrelated
- ❌ More efficient than FE if errors are random walk

In [ ]:
# Estimate First Difference
fd = pb.FirstDifferenceEstimator(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

fd_results = fd.fit()

print("="*70)
print("MODEL 5: FIRST DIFFERENCE")
print("="*70)
print(fd_results.summary())

**Interpretation**:
- Estimates effect of **changes** in X on **changes** in y
- Lost 10 observations (one per firm from differencing)
- Compare with FE to see if results are similar

---

## 4. Specification Tests {#specification-tests}

How do we choose between models? Use **specification tests**.

### 4.1 F-Test: Pooled OLS vs Fixed Effects

**Null hypothesis**: $\alpha_1 = \alpha_2 = ... = \alpha_N$ (Pooled OLS is adequate)

**Alternative**: At least one $\alpha_i$ differs (need Fixed Effects)

The F-statistic is automatically included in FE output. Let's check it:

In [ ]:
# The F-test is in the FE results
print("F-Test: Pooled OLS vs Fixed Effects")
print("="*60)
print(f"F-statistic: {fe_results.f_statistic:.4f}")
print(f"P-value: {fe_results.f_pvalue:.4e}")
print(f"\nDecision: ", end="")

if fe_results.f_pvalue < 0.05:
    print("Reject H₀ → Use Fixed Effects (firm effects are significant)")
else:
    print("Fail to reject H₀ → Pooled OLS is adequate")

### 4.2 Hausman Test: Fixed Effects vs Random Effects

**Null hypothesis**: $\mathbb{E}[\alpha_i | X_{it}] = 0$ (RE is consistent and efficient)

**Alternative**: $\mathbb{E}[\alpha_i | X_{it}] \neq 0$ (RE is inconsistent, use FE)

**Test statistic**:

$$H = (\hat{\beta}_{FE} - \hat{\beta}_{RE})' [Var(\hat{\beta}_{FE}) - Var(\hat{\beta}_{RE})]^{-1} (\hat{\beta}_{FE} - \hat{\beta}_{RE}) \sim \chi^2_k$$

In [ ]:
# Hausman Test
hausman = pb.HausmanTest(fe_results, re_results)

print("="*60)
print("HAUSMAN TEST: Fixed Effects vs Random Effects")
print("="*60)
print(hausman)

print(f"\n\nDecision: ", end="")
if hausman.pvalue < 0.05:
    print("Reject H₀ → Use Fixed Effects")
    print("(Random effects assumption violated)")
else:
    print("Fail to reject H₀ → Can use Random Effects")
    print("(RE is consistent and more efficient)")

**Practical Note**: 
- If Hausman test rejects (p < 0.05): Use **Fixed Effects**
- If Hausman test doesn't reject: Use **Random Effects** (more efficient)
- When in doubt: Use **Fixed Effects** (safer, always consistent)

---

## 5. Robust Standard Errors {#robust-se}

PanelBox supports **8 types of robust standard errors**:

1. **HC0-HC3**: Heteroskedasticity-robust (White standard errors)
2. **clustered**: Cluster-robust (by entity or time)
3. **driscoll_kraay**: Robust to spatial and temporal correlation
4. **newey_west**: HAC (heteroskedasticity and autocorrelation consistent)
5. **pcse**: Panel-corrected standard errors (Parks 1967)

Let's demonstrate the most common: **clustered standard errors**.

### 5.1 Clustered Standard Errors

In [ ]:
# Fixed Effects with clustered SE (by firm)
fe_clustered = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)

fe_clustered_results = fe_clustered.fit(cov_type='clustered', cluster_entity=True)

print("="*70)
print("FIXED EFFECTS WITH CLUSTERED STANDARD ERRORS")
print("="*70)
print(fe_clustered_results.summary())

### 5.2 Compare Different Standard Errors

Let's see how different SE types affect inference:

In [ ]:
# Estimate FE with different SE types
se_types = ['standard', 'HC0', 'HC1', 'HC2', 'HC3', 'clustered']
se_comparison = []

for se_type in se_types:
    model = pb.FixedEffects(
        formula="invest ~ value + capital",
        data=data,
        entity_col="firm",
        time_col="year"
    )
    
    if se_type == 'standard':
        results = model.fit()
    elif se_type == 'clustered':
        results = model.fit(cov_type='clustered', cluster_entity=True)
    else:
        results = model.fit(cov_type=se_type)
    
    se_comparison.append({
        'SE Type': se_type,
        'value SE': results.std_errors['value'],
        'capital SE': results.std_errors['capital']
    })

se_df = pd.DataFrame(se_comparison)
print("\nStandard Error Comparison:")
print("="*60)
print(se_df.to_string(index=False))

print("\nKey Observations:")
print("- Clustered SE are typically larger (more conservative)")
print("- HC variants adjust for heteroskedasticity")
print("- Choice matters for inference (t-stats, p-values)")

**When to use each:**

- **Standard**: Only if you're confident errors are i.i.d. (rare in panel data)
- **HC0-HC3**: Heteroskedasticity but no serial correlation
- **Clustered**: ⭐ **Recommended for panel data** (allows within-entity correlation)
- **Driscoll-Kraay**: Cross-sectional dependence (spatial correlation)
- **Newey-West**: Serial correlation with known lag structure
- **PCSE**: Panel-specific heteroskedasticity and serial correlation

---

## 6. Model Comparison {#model-comparison}

Let's create a comprehensive comparison table of all five models:

In [ ]:
# Create comparison dataframe
comparison_data = {
    'Model': ['Pooled OLS', 'Between', 'Fixed Effects', 'Random Effects', 'First Diff'],
    'value_coef': [
        pooled_results.params['value'],
        between_results.params['value'],
        fe_results.params['value'],
        re_results.params['value'],
        fd_results.params['value']
    ],
    'value_se': [
        pooled_results.std_errors['value'],
        between_results.std_errors['value'],
        fe_results.std_errors['value'],
        re_results.std_errors['value'],
        fd_results.std_errors['value']
    ],
    'capital_coef': [
        pooled_results.params['capital'],
        between_results.params['capital'],
        fe_results.params['capital'],
        re_results.params['capital'],
        fd_results.params['capital']
    ],
    'capital_se': [
        pooled_results.std_errors['capital'],
        between_results.std_errors['capital'],
        fe_results.std_errors['capital'],
        re_results.std_errors['capital'],
        fd_results.std_errors['capital']
    ],
    'R²': [
        pooled_results.rsquared,
        between_results.rsquared,
        fe_results.rsquared_within,
        re_results.rsquared,
        fd_results.rsquared
    ],
    'N_obs': [
        pooled_results.nobs,
        between_results.nobs,
        fe_results.nobs,
        re_results.nobs,
        fd_results.nobs
    ]
}

comp_df = pd.DataFrame(comparison_data)

print("="*80)
print("MODEL COMPARISON TABLE")
print("="*80)
print(comp_df.to_string(index=False))

print("\n" + "="*80)
print("NOTES:")
print("- Standard errors in parentheses")
print("- FE R² is within R² (after removing firm effects)")
print("- FD lost 10 observations from differencing")
print("- Between uses only 10 observations (firm means)")

### Visualize Coefficient Estimates

In [ ]:
# Plot coefficients across models
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

models = ['Pooled', 'Between', 'FE', 'RE', 'FD']

# Value coefficient
value_coefs = comp_df['value_coef'].values
value_ses = comp_df['value_se'].values
y_pos = np.arange(len(models))

axes[0].errorbar(value_coefs, y_pos, xerr=1.96*value_ses, fmt='o', markersize=10, capsize=5, capthick=2)
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(models)
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Coefficient Estimate', fontsize=12)
axes[0].set_title('Value Coefficient Across Models', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Capital coefficient
capital_coefs = comp_df['capital_coef'].values
capital_ses = comp_df['capital_se'].values

axes[1].errorbar(capital_coefs, y_pos, xerr=1.96*capital_ses, fmt='o', markersize=10, capsize=5, capthick=2, color='green')
axes[1].set_yticks(y_pos)
axes[1].set_yticklabels(models)
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Coefficient Estimate', fontsize=12)
axes[1].set_title('Capital Coefficient Across Models', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Observations:")
print("- Pooled OLS often differs from FE/RE (omitted variable bias)")
print("- FE and RE are often similar (but Hausman test is definitive)")
print("- FD can differ if dynamics are important")
print("- Between uses only cross-sectional variation")

---

## 7. Report Generation {#report-generation}

PanelBox can export results to HTML, Markdown, and LaTeX.

### 7.1 HTML Report

In [ ]:
# Generate HTML report (example - would save to file in practice)
print("HTML Export Example:")
print("="*60)
print("# To export to HTML:")
print("fe_results.to_html('fixed_effects_results.html')")
print("\n# This creates a professional, publication-ready HTML report")
print("# with tables, statistics, and formatting")

### 7.2 LaTeX Export (for Academic Papers)

In [ ]:
# Generate LaTeX table (example)
print("LaTeX Export Example:")
print("="*60)
print("# To export to LaTeX:")
print("fe_results.to_latex('table1.tex')")
print("\n# This creates a LaTeX table that can be directly")
print("# included in your paper with \\input{table1.tex}")
print("\n# You can also create comparison tables:")
print("from panelbox.report import create_comparison_table")
print("create_comparison_table([fe_results, re_results], 'comparison.tex')")

---

## 8. Decision Guide: Which Model to Use? {#decision-guide}

### Decision Tree:

```
Start Here
    |
    v
[Do you have panel data?]
    |
    YES --> [Are there unobserved entity effects?]
    |           |
    |           YES --> [Run F-test: Pooled vs FE]
    |           |           |
    |           |           Reject H₀ --> [Use FE or RE]
    |           |           |                  |
    |           |           |                  v
    |           |           |           [Run Hausman Test]
    |           |           |                  |
    |           |           |           Reject --> Fixed Effects ✓
    |           |           |           Don't Reject --> Random Effects ✓
    |           |           |
    |           |           Don't Reject --> Pooled OLS ✓
    |           |
    |           NO --> Pooled OLS ✓
    |
    NO --> Use standard regression
```

### Quick Reference Table:

| Situation | Recommended Model | Reason |
|-----------|------------------|--------|
| Just starting / exploratory | Pooled OLS | Baseline |
| Unobserved heterogeneity suspected | Fixed Effects | Safe, always consistent |
| Hausman test fails to reject | Random Effects | More efficient |
| Need to estimate time-invariant vars | Random Effects or Between | FE can't identify these |
| Short-run effects matter | First Difference | Focus on changes |
| Long-run effects matter | Between | Cross-sectional variation |
| Small T, large N | Random Effects | More efficient |
| Large T, small N | Fixed Effects | Asymptotic properties |
| When in doubt | Fixed Effects | Conservative choice |

### Best Practices:

1. **Always start with Fixed Effects** in applied work (unless you have strong reasons not to)
2. **Use clustered standard errors** (by entity) to be conservative
3. **Report multiple specifications** in papers (robustness)
4. **Test your assumptions** (F-test, Hausman test)
5. **Check diagnostics** (serial correlation, heteroskedasticity)
6. **Consider dynamics** (if lagged y matters, use GMM - see next notebook)

---

## Summary

In this notebook, you learned:

✅ **5 Static Panel Models**:
   - Pooled OLS (baseline)
   - Between (cross-sectional)
   - Fixed Effects (workhorse)
   - Random Effects (efficient if valid)
   - First Difference (short-run)

✅ **Specification Tests**:
   - F-test (Pooled vs FE)
   - Hausman test (FE vs RE)

✅ **Robust Standard Errors**:
   - 8 types available
   - When to use each
   - Clustered SE recommended

✅ **Model Comparison**:
   - How to compare models
   - Visualize differences
   - Interpretation

✅ **Report Generation**:
   - HTML, Markdown, LaTeX export
   - Publication-ready tables

### Next Steps:

- **[02_dynamic_gmm_complete.ipynb](./02_dynamic_gmm_complete.ipynb)**: Learn about dynamic panels and GMM
- **[03_validation_complete.ipynb](./03_validation_complete.ipynb)**: Deep dive into validation tests
- **[04_robust_inference.ipynb](./04_robust_inference.ipynb)**: Advanced inference techniques

---

*Happy modeling with PanelBox!*